In [ ]:
import numpy as np
import math

RADIUS = 50
NUM_BINS = 100
U_MAX = 5.0 / math.sqrt(RADIUS)
ALPHA = -1

In [44]:
def get_brightness(x: int, y: int) -> float:
    r = math.sqrt(x**2 + y**2)
    if (r > RADIUS):
        return 0.0
    if (r == 0.0):
        return math.inf
    return (r**ALPHA)

In [45]:
# each coordinate represents the corresponding brightness coordinate on the disk
brightness_grid = np.zeros((RADIUS*2, RADIUS*2))

for i in range(-RADIUS, RADIUS):
    for j in range(-RADIUS, RADIUS):
        brightness_grid[i+RADIUS, j+RADIUS] = get_brightness(i, j)

print(brightness_grid)


# u_intensities = np.linspace(-U_MAX, U_MAX, NUM_BINS)


[[0.         0.         0.         0.33333333 0.         0.        ]
 [0.         0.35355339 0.4472136  0.5        0.4472136  0.35355339]
 [0.         0.4472136  0.70710678 1.         0.70710678 0.4472136 ]
 [0.33333333 0.5        1.                inf 1.         0.5       ]
 [0.         0.4472136  0.70710678 1.         0.70710678 0.4472136 ]
 [0.         0.35355339 0.4472136  0.5        0.4472136  0.35355339]]
